In [6]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from itertools import combinations 
import plotnine as p

## Randomized Control Trial (RCT):

- Key Idea: Remember that our goal is to find clever ways to remove bias and make the treatment group and the control group comparable. Thus all the differences that we see will be only the average effect of the applied treatment. So, we require to make association be causation:

$$ E[Y|T=1] - E[Y|T=0] = \underbrace{E[Y_1 - Y_0|T=1]}_{ATET} + \underbrace{E[Y_0|T=1] - E[Y_0|T=0]}_{BIAS} $$
    

The first tool we have to make the bias vanish is **Randomized Experiments**.

In short, an RCT is an study design that **randomly assigns** participants into an experimental group or a control group. As the study is conducted, the **only expected difference** between the control and experimental groups in a RCT is the outcome variable being studied.

Randomisation annihilates bias by making the potential outcomes independent of the treatment: 

$$ (Y_0, Y_1) \,\bot\, T  $$

Saying that the potential outcomes are independent of the treatment is saying that they would be, in expectation, the same in the treatment or the control group. In simpler terms, it means that treatment and control groups are comparable.

Therefore, this means that the treatment is the only thing generating a difference between the outcome in the treated and in the control group.

### Example 1: the ideal experiment

In 2020, the Coronavirus Pandemic forced businesses to adapt to social distancing. In this context, we want to answer if online learning has a negative or positive impact on the student’s academic performance.

To solve that, we need to make the treated and untreated comparable. One way to force this is by randomly assigning the online and presential classes to students.

Imagine that we've randomized classes: some students were assigned to have face-to-face lectures, others to have only online lessons, and a third group to have a blended format of both online and face-to-face classes. Then, we collect data on a standard exam at the end of the semester:

In [7]:
data = pd.read_csv('https://github.com/matheusfacure/python-causality-handbook/raw/master/causal-inference-for-the-brave-and-true/data/online_classroom.csv')
print(data.shape)
data.head()


(323, 10)


gender  asian  black  hawaiian  hispanic  unknown  white  format_ol  \
0       0    0.0    0.0       0.0       0.0      0.0    1.0          0   
1       1    0.0    0.0       0.0       0.0      0.0    1.0          0   
2       1    0.0    0.0       0.0       0.0      0.0    1.0          0   
3       1    0.0    0.0       0.0       0.0      0.0    1.0          0   
4       1    0.0    0.0       0.0       0.0      0.0    1.0          1   

   format_blended  falsexam  
0             0.0  63.29997  
1             0.0  79.96000  
2             1.0  83.37000  
3             1.0  90.01994  
4             0.0  83.30000

To estimate the causal effect, we can simply compute the mean score for each of the treatment groups.

In [8]:
(data
 .assign(class_format = np.select(
     [data["format_ol"].astype(bool), data["format_blended"].astype(bool)],
     ["online", "blended"],
     default="face_to_face" #create a new variable
 ))
 .groupby(["class_format"]) #group by the new variable (treatments)
 .mean()) #get the exam's mean


gender     asian     black  hawaiian  hispanic   unknown  \
class_format                                                               
blended       0.550459  0.217949  0.102564  0.025641  0.012821  0.012821   
face_to_face  0.633333  0.202020  0.070707  0.000000  0.010101  0.000000   
online        0.542553  0.228571  0.028571  0.014286  0.028571  0.000000   

                 white  format_ol  format_blended   falsexam  
class_format                                                  
blended       0.628205        0.0             1.0  77.093731  
face_to_face  0.717172        0.0             0.0  78.547485  
online        0.700000        1.0             0.0  73.635263

We can see that face-to-face classes yield a 78.54 average score, while online courses yield a 73.63 average score. Not so good news for the proponents of online learning. The ATT for an online class is thus -4.91. This means that online classes cause students to perform about 5 points lower, on average. 

A good sanity check to see if the randomization was done right (or if you are looking at the correct data) is to check if the treated are equal to the untreated in pre-treatment variables. Our data has information on gender and ethnicity to see if they are similar across groups. We can say that they look pretty similar for the gender, asian, hispanic, and white variables. The black variable, however, seems a little bit different.

### Example 2: the Methodology of Fisher’s sharp null

The main advantage of randomization inference is that it allows us to make probability calculations revealing whether the data are likely a draw from a truly random distribution or not.

**Fisher’s sharp null** is a claim we make wherein no unit in our data, when treated, had a causal effect. The value of Fisher’s sharp null is that it allows us to make an “exact” inference that does not depend on hypothesized distributions.


### Methodological steps for an RCT:

There are six steps to randomization inference: (1) the choice of the sharp null, (2) the construction of the null, (3) the picking of a different treatment vector, (4) the calculation of the corresponding test statistic for that new treatment vector, (5) the randomization over step 3 as you cycle through a number of new treatment vectors (ideally all possible combinations), and (6) the calculation the exact p-value.

Imagine that you work for a homeless shelter with a cognitive behavioral therapy (CBT) program for treating mental illness and substance abuse. You have enough funding to enlist four people into the study, but you have eight residents. Therefore, there are four in treatment and four in control. After concluding the CBT, residents are interviewed to determine the severity of their mental illness symptoms.

In [9]:
def read_data(file): 
    return pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/" + file)

ri = read_data('ri.dta')
ri


name  d   y  y0  y1
0    Andy  1  10   .  10
1     Ben  1   5   .   5
2    Chad  1  16   .  16
3  Daniel  1   3   .   3
4   Edith  0   5   5   .
5   Frank  0   7   7   .
6  George  0   8   8   .
7    Hank  0  10  10   .

To calculate the exact p-value in this example, it is necessary to create the assignment of values of being able to be treated ($D = 1$) or not be treated ($D = 0$), from them take the average of the potential results and obtaining the result $Y(D=1) - Y(D=0)$, repeat this procedure until the combinations of $D$ can no longer be repeated.

Finally put the results in the following formula:

$$\Pr\Big(t(D',Y)\geq t(D_{observed},Y) \mid \delta_i=0, \forall i)\Big)= \dfrac{\sum_{D'\in \Omega } I(t(D',Y) \geq t(D_{observed},Y))}{K}$$

The following code illustrates the procedure performed manually

In [10]:
# data 1: grupos de control y tratamiento
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
def read_data(file): 
    return pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/" + file)

ri = read_data('ri.dta') # leer data
ri['id'] = range(1,9) # colocar id
treated = range(1,5) # numero de elementos tratados


#data 2: posibles combinaciónes
combo = pd.DataFrame(np.array(list(combinations(ri['id'], 4))), 
                     columns=['treated1', 'treated2', 'treated3', 'treated4']) # fijar 4 casos tratados
combo['permutation'] = np.arange(1,71) # combinatoria de 8 elementos en 4 grupos (70)


# combinar dos datas:
combo['key'] = 1 #variable para combinar dos datas (key)
ri['key'] = 1
combo2 = pd.merge(ri, combo, on='key')

combo2 = combo2.sort_values(['permutation', 'name']) # ordenar data

# Permutaciones: aleatorizar el tratamiento 
combo2['d'] = 0
combo2.loc[(combo2.treated1==combo2.id) | 
          (combo2.treated2==combo2.id) | 
          (combo2.treated3==combo2.id) | 
          (combo2.treated4==combo2.id), 'd'] = 1

# media de resultados por grupo de tratamiento y control
te1 = combo2[combo2.d==1].groupby('permutation')['y'].mean() # media tratamiento
te0 = combo2[combo2.d==0].groupby('permutation')['y'].mean() # media control

# ATE
p_value = pd.merge(te1, te0, how='inner', on="permutation")
p_value.columns = ['te1', 'te0']
p_value = p_value.reset_index()
p_value['ate'] = p_value['te1'] - p_value['te0']
p_value = p_value.sort_values(by='ate', ascending=False)

# rank
p_value['rank'] = range(1, p_value.shape[0]+1)
p_value = p_value[p_value['permutation'] == 1]
x = p_value['rank'] / 70 # p-valor
x



0    0.371429
Name: rank, dtype: float64